In [0]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import learning_curve, GridSearchCV, RandomizedSearchCV, train_test_split
from imutils import paths
import numpy as np
import argparse
import imutils
import time
import cv2
import os

In [0]:
def extract_color_histogram(image, bins=(8, 8, 8)):
	# extract a 3D color histogram from the HSV color space using
	# the supplied number of `bins` per channel
	hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
	hist = cv2.calcHist([hsv], [0, 1, 2], None, bins,
		[0, 180, 0, 256, 0, 256])

	# handle normalizing the histogram if we are using OpenCV 2.4.X
	if imutils.is_cv2():
		hist = cv2.normalize(hist)

	# otherwise, perform "in place" normalization in OpenCV 3 (I
	# personally hate the way this is done
	else:
		cv2.normalize(hist, hist)

	# return the flattened histogram as the feature vector
	return hist.flatten()

In [3]:
pwd


'/content'

In [0]:
cd /content/kaggle_dogs_vs_cats

[Errno 2] No such file or directory: '/content/kaggle_dogs_vs_cats'
/content


In [4]:
!pip install pyunpack
!pip install patool

  Created wheel for pyunpack: filename=pyunpack-0.1.2-cp36-none-any.whl size=5162 sha256=efb08f37256279ffab87dc5f5feb5f962c7e92756432251044966e30a114e845
  Stored in directory: /root/.cache/pip/wheels/af/44/08/60613970881e542c0baad1f2dea5ed8e6716bc573f49197b7e
Successfully built pyunpack
     |████████████████████████████████| 81kB 2.8MB/s 


In [0]:
from pyunpack import Archive
Archive("train.rar").extractall(".")

In [6]:
cd /content/train

/content/train


In [7]:
ls

cat.0.jpg      cat.10709.jpg  cat.11417.jpg  cat.12125.jpg  cat.1584.jpg
cat.10000.jpg  cat.1070.jpg   cat.11418.jpg  cat.12126.jpg  cat.1585.jpg
cat.10001.jpg  cat.10710.jpg  cat.11419.jpg  cat.12127.jpg  cat.1586.jpg
cat.10002.jpg  cat.10711.jpg  cat.1141.jpg   cat.12128.jpg  cat.1587.jpg
cat.10003.jpg  cat.10712.jpg  cat.11420.jpg  cat.12129.jpg  cat.1588.jpg
cat.10004.jpg  cat.10713.jpg  cat.11421.jpg  cat.1212.jpg   cat.1589.jpg
cat.10005.jpg  cat.10714.jpg  cat.11422.jpg  cat.12130.jpg  cat.158.jpg
cat.10006.jpg  cat.10715.jpg  cat.11423.jpg  cat.12131.jpg  cat.1590.jpg
cat.10007.jpg  cat.10716.jpg  cat.11424.jpg  cat.12132.jpg  cat.1591.jpg
cat.10008.jpg  cat.10717.jpg  cat.11425.jpg  cat.12133.jpg  cat.1592.jpg
cat.10009.jpg  cat.10718.jpg  cat.11426.jpg  cat.12134.jpg  cat.1593.jpg
cat.1000.jpg   cat.10719.jpg  cat.11427.jpg  cat.12135.jpg  cat.1594.jpg
cat.10010.jpg  cat.1071.jpg   cat.11428.jpg  cat.12136.jpg  cat.1595.jpg
cat.10011.jpg  cat.10720.jpg  cat.11429.jpg  cat.121

In [12]:
cd /content/

/content


In [0]:
#dataset="kaggle_dogs_vs_cats"
dataset="train"
jobs=-1 #(-1 uses all available cores)
imagePaths = list(paths.list_images(dataset))
# initialize the data matrix and labels list
data = []
labels = []

In [14]:
for (i, imagePath) in enumerate(imagePaths):
	# load the image and extract the class label (assuming that our
	# path as the format: /path/to/dataset/{class}.{image_num}.jpg
	image = cv2.imread(imagePath)
	label = imagePath.split(os.path.sep)[-1].split(".")[0]

	# extract a color histogram from the image, then update the
	# data matrix and labels list
	hist = extract_color_histogram(image)
	data.append(hist)
	labels.append(label)

	# show an update every 1,000 images
	if i > 0 and i % 1000 == 0:
		print("[INFO] processed {}/{}".format(i, len(imagePaths)))


[INFO] processed 1000/25000
[INFO] processed 2000/25000
[INFO] processed 3000/25000
[INFO] processed 4000/25000
[INFO] processed 5000/25000
[INFO] processed 6000/25000
[INFO] processed 7000/25000
[INFO] processed 8000/25000
[INFO] processed 9000/25000
[INFO] processed 10000/25000
[INFO] processed 11000/25000
[INFO] processed 12000/25000
[INFO] processed 13000/25000
[INFO] processed 14000/25000
[INFO] processed 15000/25000
[INFO] processed 16000/25000
[INFO] processed 17000/25000
[INFO] processed 18000/25000
[INFO] processed 19000/25000
[INFO] processed 20000/25000
[INFO] processed 21000/25000
[INFO] processed 22000/25000
[INFO] processed 23000/25000
[INFO] processed 24000/25000


In [15]:
print("[INFO] constructing training/testing split...")
(trainData, testData, trainLabels, testLabels) = train_test_split(
	data, labels, test_size=0.25, random_state=42)

# construct the set of hyperparameters to tune
params = {"n_neighbors": np.arange(1, 31, 2),
	"metric": ["euclidean", "cityblock"]}

# tune the hyperparameters via a cross-validated grid search
print("[INFO] tuning hyperparameters via grid search")
model = KNeighborsClassifier(jobs)
grid = GridSearchCV(model, params)
start = time.time()
grid.fit(trainData, trainLabels)
end= time.time() - start
# evaluate the best grid searched model on the testing data
print("[INFO] grid search took {:.2f} seconds".format(end))
acc = grid.score(testData, testLabels)
print("[INFO] grid search accuracy: {:.2f}%".format(acc * 100))
print("[INFO] grid search best parameters: {}".format(
	grid.best_params_))

[INFO] constructing training/testing split...
[INFO] tuning hyperparameters via grid search


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


[INFO] grid search took 8195.66 seconds
[INFO] grid search accuracy: 64.42%
[INFO] grid search best parameters: {'metric': 'cityblock', 'n_neighbors': 27}
[INFO] constructing training/testing split...
[INFO] tuning hyperparameters via grid search


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


[INFO] grid search took 8195.66 seconds
[INFO] grid search accuracy: 64.42%
[INFO] grid search best parameters: {'metric': 'cityblock', 'n_neighbors': 27}


In [16]:
# tune the hyperparameters via a randomized search
grid = RandomizedSearchCV(model, params)
start = time.time()
grid.fit(trainData, trainLabels)

# evaluate the best randomized searched model on the testing
# data
print("[INFO] randomized search took {:.2f} seconds".format(
	time.time() - start))
acc = grid.score(testData, testLabels)
print("[INFO] grid search accuracy: {:.2f}%".format(acc * 100))
print("[INFO] randomized search best parameters: {}".format(
	grid.best_params_))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


[INFO] randomized search took 2774.03 seconds
[INFO] grid search accuracy: 64.32%
[INFO] randomized search best parameters: {'n_neighbors': 21, 'metric': 'cityblock'}
